In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from matplotlib import rcParams
from matplotlib.colors import LogNorm
from pymatsolver import Pardiso
from SimPEG import EM, Maps, Mesh, Utils

rcParams['font.size'] = 16


In [4]:
# creat 3D mesh
cs, ncx, ncy, ncz, npad = 20., 10, 10, 10, 17
pad_rate = 1.3
hx = [(cs,npad,-pad_rate), (cs,ncx), (cs,npad,pad_rate)]
hy = [(cs,npad,-pad_rate), (cs,ncy), (cs,npad,pad_rate)]
hz = [(cs,npad,-pad_rate), (cs,ncz), (cs,npad,pad_rate)]
mesh = Mesh.TensorMesh([hx,hy,hz], 'CCC')

# mesh imformation
print(mesh)



  TensorMesh: 85,184 cells

                      MESH EXTENT             CELL WIDTH      FACTOR
  dir    nC        min           max         min       max      max
  ---   ---  ---------------------------  ------------------  ------
   x     44     -7,510.36      7,510.36     20.00  1,730.08    1.30
   y     44     -7,510.36      7,510.36     20.00  1,730.08    1.30
   z     44     -7,510.36      7,510.36     20.00  1,730.08    1.30




In [5]:
# model parameters
sig_full = 1e-3
sig_air = 1e-8

sigma = np.ones(mesh.nC)*sig_full
#sigma[mesh.gridCC[:,2]>0.] = sig_air

# print diffusion distance and make sure mesh padding goes beyond that
print("maximum diffusion distance: {:1.4e} m".format(1260*np.sqrt(1e3*2.5e-3)))
print("minimum diffusion distance: {:1.4e} m".format(1260*np.sqrt(1e3*5e-5)))


maximum diffusion distance: 1.9922e+03 m
minimum diffusion distance: 2.8174e+02 m


In [ ]:
indx = 33
indy = 33
indz = 33

fig, ax = plt.subplots(1,3, figsize = (18, 6))

clim = [1e-8, 1e-1]

xlim = [-200., 200.]
ylim = [-200., 200.]

dat1 = mesh.plotSlice(sigma, grid=True, ax=ax[0], ind=indz, pcolorOpts={'norm':LogNorm()}, clim=clim)
dat2 = mesh.plotSlice(sigma, grid=True, ax=ax[1], ind=indy, normal='Y', pcolorOpts={'norm':LogNorm()}, clim=clim)
dat3 = mesh.plotSlice(sigma, grid=True, ax=ax[2], ind=indx, normal='X', pcolorOpts={'norm':LogNorm()}, clim=clim)

cb = plt.colorbar(dat3[0], orientation="horizontal", ax = ax[0])

ax[0].set_xlim(xlim)
ax[0].set_ylim(ylim)
ax[1].set_xlim(xlim)
ax[1].set_ylim(ylim)
ax[2].set_xlim(xlim)
ax[2].set_ylim(ylim)

plt.tight_layout()

ax[0].set_aspect(1)
ax[1].set_aspect(1)
ax[2].set_aspect(1)

ax[0].set_title("Conductivity at Z={:1.0f} m".format(mesh.vectorCCz[indz]))
ax[1].set_title("Y= {:1.0f}m".format(mesh.vectorCCy[indy]))
ax[2].set_title("X= {:1.0f}m".format(mesh.vectorCCx[indx]))

cb.set_label("$\sigma$ (S/m)")

#fig.savefig('./data/model_half/model.png', dpi=350)



In [6]:
# assemble the sources and receivers
time = np.logspace(np.log10(5e-5), np.log10(2.5e-3), 21)

srcList = []

location = np.array([[0., 0., 0.]])
rx_z = EM.TDEM.Rx.Point_dbdt(location, time, 'z')
src = EM.TDEM.Src.CircularLoop([rx_z], orientation='z', loc=location)
srcList.append(src)


In [9]:
timesteps1 = [(5e-7, 10), (5*5e-7, 20), (5**2*5e-7, 20), (5**3*5e-7, 20), (5**4*5e-7, 10)]

timesteps2 = [(5*5e-8, 20), (5**2*5e-8, 40), (5**3*5e-8, 40), (5**4*5e-8, 40), (5**5*5e-8, 20)]

timesteps3 = [(5**2*5e-9, 40), (5**3*5e-9, 80), (5**4*5e-9, 80), (5**5*5e-9, 80), (5**6*5e-9, 40)]

timesteps = [(5**3*5e-10, 80), (5**4*5e-10, 160), (5**5*5e-10, 160), (5**6*5e-10, 160), (5**7*5e-10, 80)]

prb = EM.TDEM.Problem3D_e(mesh, Solver=Pardiso, verbose=True, timeSteps=timesteps, sigmaMap=Maps.IdentityMap(mesh))
survey = EM.TDEM.Survey(srcList)
survey.pair(prb)


AttributeError: module 'pymatsolver' has no attribute 'direct'

In [ ]:
fields = prb.fields(sigma)


In [ ]:
dpred = survey.dpred(sigma, f=fields)
DPRED = dpred.reshape((survey.nSrc, rx_z.times.size))
DPRED = DPRED.T

#np.save('E:\Mycloud\Code\TEM_3D_test\TEM_Inv_3D\data\model_full\dpred_tp4.npy', dpred)


In [ ]:

fig = plt.figure(figsize=(9, 4), dpi=350)
plt.loglog(time, -DPRED, 'k.-', lw=1)
plt.xlabel("t (s)")
plt.ylabel("Voltage (V/Am$^2$)")
plt.grid(which="both", alpha=0.2)

